In [2]:
import numpy as np
import umap    
import umap.plot
import torch
from torch.utils.data import DataLoader

from pathlib import Path
import numpy as np
import pandas as pd
import sentence_transformers.losses as losses
from sentence_transformers import SentenceTransformer, SimilarityFunction
from sentence_transformers.trainer import SentenceTransformerTrainer, SentenceTransformerTrainingArguments
from sentence_transformers.training_args import BatchSamplers
from datasets import Dataset
import sys
import os
import logging
import re

SCRIPT_DIR = Path.cwd().parent
sys.path.insert(0, str(SCRIPT_DIR))

from embeddings.embedding_library import EmbeddingLibrary

In [3]:
path_to_papers = Path(r'../data/papers')
path_to_embs = Path(r'../data/embeddings')

# should all be lower case
end_of_paper_words = [
    'references',
    'acknowledgement', 
    'appendix',
]

papers_to_skip = [
    '2311.11329v2_content.txt',
    '2411.09324v2_content.txt',
    '2411.14259v1_content.txt',
    '2309.01837v3_content.txt'
    #'2403.12778v2_content.txt' 
]

In [4]:
model_name = 'all-distilroberta-v1'
device = 'cuda'

In [5]:
model = SentenceTransformer(model_name, device=device)

# https://huggingface.co/sentence-transformers/all-distilroberta-v1

In [ ]:
path_to_weights = r'C:\Users\rooty\UWEC\Fall 24\CS 426\GroupProject\GroupProject\scholarly-search\finetune\semsim'

model = SentenceTransformer(path_to_weights, device=device)

In [20]:
emb_lib = EmbeddingLibrary(
    path_to_papers=path_to_papers,
    path_to_embs=path_to_embs,
    model=model,
    end_of_paper_words=end_of_paper_words,
    papers_to_skip=papers_to_skip,
    norm_embs=True,
    name=model_name,
    log_lvl=logging.INFO,
    path_to_log=Path("../log")
)

model.similarity_fn_name = SimilarityFunction.COSINE # TODO fix

Batches: 100%|██████████| 1/1 [00:00<00:00, 14.15it/s]


In [5]:
emb_lib.multi_sec_embed(skip_existing=False)

Batches: 100%|██████████| 2/2 [00:00<00:00,  7.09it/s]


### Test Embedding Lib

In [21]:
embs = emb_lib.paper_embs

In [9]:
mapper = umap.UMAP().fit(embs)
p = umap.plot.points(mapper, labels=pd.Series([i for i in range(embs.shape[0])]), show_legend=False)
umap.plot.show(p)

In [22]:
model.similarity(embs, embs)

tensor([[1.0000, 0.1973, 0.3841,  ..., 0.5426, 0.3374, 0.3162],
        [0.1973, 1.0000, 0.3558,  ..., 0.2475, 0.3143, 0.4754],
        [0.3841, 0.3558, 1.0000,  ..., 0.3466, 0.4973, 0.3639],
        ...,
        [0.5426, 0.2475, 0.3466,  ..., 1.0000, 0.3382, 0.3323],
        [0.3374, 0.3143, 0.4973,  ..., 0.3382, 1.0000, 0.2821],
        [0.3162, 0.4754, 0.3639,  ..., 0.3323, 0.2821, 1.0000]])

In [9]:
# Paper: 2311.00207v3

prompt = ""
response = '''SECTION: IIntroduction

Next-generation (NextG) networks promise to support ultra-reliable and low-latency communication for rapidly evolving wireless devices[29]. Emerging networks are thus challenged to establish new features (e.g., adaptive coding and enhanced modulation) to overcome rapidly changing channel conditions and to achieve more efficient use of spectrum[102,63].
Machine Learning (ML) overcomes this barrier by revolutionizing the entire wireless network protocol stack[67].'''

emb_lib.search_papers(prompt, response)

['Next-generation (NextG) networks promise to support ultra-reliable and low-latency communication for rapidly evolving wireless devices. Emerging networks are thus challenged to establish new features (e.g., adaptive coding and enhanced modulation) to overcome rapidly changing channel conditions and to achieve more efficient use of spectrum.', 'Machine Learning (ML) overcomes this barrier by revolutionizing the entire wireless network protocol stack.', 'Recent research introduces joint source-channel coding (JSCC), an end-to-end wireless communication system leveraging deep neural networks (DNNs) for both transmitter and receiver. This ML approach jointly optimizes source and channel coding in a cross-layer framework to handle diverse and challenging channel conditions.', 'To effectively cope with the multipath fading effects, the JSCC-encoded data can be further modulated into continuous signal waveforms through orthogonal frequency division multiplexing (OFDM).', 'The DNN models for

Batches: 100%|██████████| 1/1 [00:00<00:00,  4.31it/s]


['2311.00207v3',
 '2406.08298v5',
 '2411.09101v1',
 '2410.23279v3',
 '2411.13037v1']

In [ ]:
# from 2411.06091

prompt = ""
response = '''Remote sensing technology, employing aerial or satellite platforms, facilitates the acquisition of earth observation data from afar, marking its utility across diverse sectors such as agriculture monitoring, environmental surveillance evaluation, urban planning, and disaster management. At the heart of these applications lies the interpretation of remote sensing imagery, a task that has been considerably challenging[1]. The advent of deep learning technology[2]has significantly enhanced the precision and automation of remote sensing image analysis, encompassing object detection, land cover classification, and change detection. Nonetheless, the intrinsic complexity of remote sensing scenarios—attributable to variations in sensor technology, atmospheric conditions, and image resolution—results in the performance disparity of analytical models across different contexts. The prohibitive costs associated with manual annotation further exacerbate this issue, rendering the re-labeling of samples for specific scenarios impractical. Consequently, there is a growing demand for models endowed with superior generalization capabilities, capable of adeptly navigating the multifaceted landscape of remote sensing applications.

Seeking a method that discerns both the semantic information of different scene images and the semantic details within each area of the images is essential for effectively processing scenographic remote sensing imagery. It has been observed that scenographic images in remote sensing typically exhibit specific distribution patterns, where natural elements (such as mountains, bodies of water, forests, lakes, and grasslands) and man-made elements (such as buildings, farmlands, plowed lands, and gardens) often display pronounced clustering. This tendency for feature aggregation leads to the formation of discernible geographic pattern clusters among individual image patches and their neighbors, thereby establishing intricate spatial relationships across the varying patches and their adjacent areas.'''

emb_lib.search_papers(prompt, response)

**These results are sub-par**

### Finetuning

In [23]:
# create paper chunks and Dataset

paper_chunks = []

for paper_path in emb_lib.paper_paths:
    with open(paper_path, 'r', encoding='utf-8') as p:
        paper = p.read()
        title, chunks = emb_lib.preprocess_paper(paper)
        paper_chunks.extend(chunks)
        paper_chunks.append(title)

dataset = Dataset.from_dict({
    "anchor": paper_chunks,
    "positive": paper_chunks,
})

['The digital image forensics based research works in literature classifying natural and computer generated images primarily focuses on binary tasks. These tasks typically involve the classification ofnatural images versus computer graphics imagesonly ornatural images versus GAN generated imagesonly, but not natural images versus both types of generated images simultaneously. Furthermore, despite the support of advanced convolutional neural networks and transformer based architectures that can achieve impressive classification accuracies for this forensic classification task of distinguishing natural and computer generated images, these models are seen to fail over the images that have undergone post-processing operations intended to deceive forensic algorithms, such as JPEG compression, Gaussian noise addition, etc. In this digital image forensic based work to distinguish between natural and computer-generated images encompassing both computer graphics and GAN generated images, we pro

In [25]:
# train a tokenizer

batch_size = 1000
all_texts = np.random.shuffle([dataset[i : i + batch_size]["anchor"] for i in range(0, len(dataset), batch_size)])

def batch_iterator():
    for i in range(0, len(dataset), batch_size):
        yield dataset[i : i + batch_size]["anchor"]

In [26]:
model.tokenizer = model.tokenizer.train_new_from_iterator(batch_iterator(), vocab_size=55000)

In [27]:
# set dropout in model

from torch.nn.modules.dropout import Dropout

def set_dropout(model, p):

    def set_d(module):
        for c in module.children():
            if len(list(module.children())) == 0: return
            if isinstance(c, Dropout):
                c.p = p
            set_d(c)

    for m in model.modules():
        set_d(m)

set_dropout(model, 0.5)   

In [29]:
# Use the MultipleNegativesRankingLoss
train_loss = losses.MultipleNegativesRankingLoss(model)

args = SentenceTransformerTrainingArguments(
    output_dir=f"training/{model_name}",
    num_train_epochs=5,
    per_device_train_batch_size=16,
    #per_device_eval_batch_size=16,
    warmup_ratio=0.1,
    fp16=True,  # Set to False if your GPU can't handle FP16
    bf16=False,  # Set to True if your GPU supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # Losses using "in-batch negatives" benefit from no duplicates
    # Optional tracking/debugging parameters:
    # eval_strategy="steps",
    # eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    logging_steps=100,
    run_name=f"{model_name}",  # Used in W&B if `wandb` is installed
)



In [30]:
trainer = SentenceTransformerTrainer(
    model = model,
    args = args,
    train_dataset = dataset,
    loss = train_loss,
    tokenizer = model.tokenizer,  
)
trainer.train()

In [31]:
path_to_weights = r'C:\Users\rooty\UWEC\Fall 24\CS 426\GroupProject\GroupProject\scholarly-search\weights\semsim'
model.save(path_to_weights)

                                                  
 34%|███▎      | 518/1535 [05:09<05:38,  3.01it/s]

{'loss': 0.454, 'grad_norm': 5.565038204193115, 'learning_rate': 3.14935064935065e-05, 'epoch': 0.33}




                                                  
 34%|███▎      | 518/1535 [06:17<05:38,  3.01it/s]

{'loss': 0.0543, 'grad_norm': 3.9539802074432373, 'learning_rate': 4.844315713251267e-05, 'epoch': 0.65}


                                                  
 34%|███▎      | 518/1535 [06:59<05:38,  3.01it/s]

...

                                                  
 34%|███▎      | 518/1535 [17:48<05:38,  3.01it/s] 

{'loss': 0.0063, 'grad_norm': 1.005142092704773, 'learning_rate': 1.3758146270818247e-06, 'epoch': 4.89}


                                                  
100%|██████████| 1535/1535 [13:45<00:00,  1.86it/s]

{'train_runtime': 825.2674, 'train_samples_per_second': 29.706, 'train_steps_per_second': 1.86, 'train_loss': 0.04364390678048522, 'epoch': 5.0}


TrainOutput(global_step=1535, training_loss=0.04364390678048522, metrics={'train_runtime': 825.2674, 'train_samples_per_second': 29.706, 'train_steps_per_second': 1.86, 'total_flos': 0.0, 'train_loss': 0.04364390678048522, 'epoch': 5.0})